In [1]:
import os
import numpy as np
import mne
from scipy import stats
import scipy.io
import h5py
from scipy.signal import butter, filtfilt

mne.set_log_level('error')

from utils.load import Load
from config.default import cfg

%load_ext autoreload
%autoreload 2


In [2]:
loader = Load(cfg)

In [3]:
# Change this one-by-one to process all subjects in the dataset
# Yeah, Im lazy, shut up!
subject_id = 0

In [9]:
raw_runs = loader.load_subject(subject_id = subject_id)

In [10]:
preprocessed_runs = raw_runs.copy()
for run in preprocessed_runs:
    run = run.resample(100)
    run = run.drop_channels(cfg['bad_channels'][cfg['subjects'][subject_id]])
    run = run.set_eeg_reference('average', projection=False)

    not_ROI = [x for x in cfg['not_ROI_channels'] if x not in cfg['bad_channels'][cfg['subjects'][subject_id]]] # Dont drop channels twice
    run = run.drop_channels(not_ROI)                  


In [78]:
def bandpass_filter(data, low, high, fs, order=5):
    nyq = 0.5 * fs
    low /= nyq
    high /= nyq
    b, a = scipy.signal.butter(order, [low, high], btype="band")
    return scipy.signal.lfilter(b, a, data)

def sliding_window(data, window_size, step_size):
    return np.array([data[i:i + window_size] for i in range(0, len(data) - window_size + 1, step_size)])

def power_spectral_density(data, fs):
    freqs, psd = scipy.signal.welch(data, fs, nperseg=len(data), scaling="density")
    return freqs, psd

def extract_band_psd(freqs, psd, low, high):
    return psd[np.logical_and(freqs >= low, freqs <= high)]

def extract_features_std_mean(eeg_data, fs=250, window_size=250, step_size=125):
    num_channels, num_samples = eeg_data.shape

    # Bandpass filter to isolate Mu and Beta rhythms (8-30 Hz)
    filtered_data = np.zeros((num_channels, num_samples))
    for i in range(num_channels):
        filtered_data[i] = bandpass_filter(eeg_data[i], 8, 30, fs)

    # Apply sliding window
    windowed_data = []
    for i in range(num_channels):
        windowed_data.append(sliding_window(filtered_data[i], window_size, step_size))
    windowed_data = np.array(windowed_data)


    # Calculate Power Spectral Density (PSD) for each window
    num_windows = windowed_data.shape[1]
    psd_features = np.zeros((num_channels, num_windows, 8))
    for ch in range(num_channels):
        for win in range(num_windows):
            freqs, psd = power_spectral_density(windowed_data[ch, win], fs)
            freqs = freqs[freqs >= 0]
            psd = psd[:len(freqs)]
            
            
            log_psd = np.log10(psd)

            

            # Separate PSD for Mu and Beta bands
            mu1_psd = extract_band_psd(freqs, psd, 8, 10)
            mu2_psd = extract_band_psd(freqs, psd, 10, 12)
            beta1_psd = extract_band_psd(freqs, psd, 13, 21)
            beta2_psd = extract_band_psd(freqs, psd, 21, 30)
            
            # Calculate the mean and standard deviation for each band
            mu1_mean, mu1_std = np.mean(mu1_psd), np.std(mu1_psd)
            mu2_mean, mu2_std = np.mean(mu2_psd), np.std(mu2_psd)
            beta1_mean, beta1_std = np.mean(beta1_psd), np.std(beta1_psd)
            beta2_mean, beta2_std = np.mean(beta2_psd), np.std(beta2_psd)
            
            psd_features[ch, win] = [mu1_mean, mu2_mean, mu1_std, mu2_std, beta1_mean, beta2_mean, beta1_std, beta2_std]

    return psd_features


def extract_features(eeg_data, fs=250, window_size=250, step_size=125):
    num_channels, num_samples = eeg_data.shape

    # Bandpass filter to isolate Mu and Beta rhythms (8-30 Hz)
    filtered_data = np.zeros((num_channels, num_samples))
    for i in range(num_channels):
        filtered_data[i] = bandpass_filter(eeg_data[i], 8, 30, fs)

    # Apply sliding window
    windowed_data = []
    for i in range(num_channels):
        windowed_data.append(sliding_window(filtered_data[i], window_size, step_size))
    windowed_data = np.array(windowed_data)

    # Calculate Power Spectral Density (PSD) for each window
    num_windows = windowed_data.shape[1]
    psd_features = []
    for ch in range(num_channels):
        channel_psd_features = []
        for win in range(num_windows):
            freqs, psd = power_spectral_density(windowed_data[ch, win], fs)
            freqs = freqs[freqs >= 0]
            psd = psd[:len(freqs)]

            # Separate PSD for Mu and Beta bands
            psd = extract_band_psd(freqs, psd, 8, 30)
            
            
            channel_psd_features.append(psd)
        psd_features.append(channel_psd_features)

    return psd_features


To improve the feature extraction for better SVM performance, consider the following modifications to the feature extraction process:

1. **Extract more features**: In addition to the mean and standard deviation of the PSD, extract other features such as the median, skewness, and kurtosis of the PSD in each frequency band.

2. **Relative Power**: Compute the relative power of the Mu and Beta bands, which is the ratio of the power in the specific band to the total power across all frequency bands.

3. **Spectral edge frequency**: Compute the spectral edge frequency, which is the frequency below which a certain percentage (e.g., 95%) of the total power of the spectrum is contained. This feature can provide information about the concentration of power within the frequency bands.


In [79]:
sfreq = raw_runs[0].info['sfreq']
print(f'Freq: {sfreq}')

tmin = -0.5
tmax = 7



features = {'thumb': [], 'index': [], 'middle': [], 'ring': [], 'little': []}
for i in range(len(preprocessed_runs)):
    events, _ = mne.events_from_annotations(preprocessed_runs[i])
    print(f'Processing run {i+1} of {len(preprocessed_runs)}')
    for trigger in events:
        time, _, label = trigger
        if label in [2, 3, 4, 5, 6]: # Drop 'No instruction' and 'Rest' events
            # Epoching
            data = preprocessed_runs[i].get_data()[...,time+int(tmin*sfreq):time+int(tmax*sfreq)] 

            psd = extract_features(data, fs = sfreq, window_size = 50, step_size = 50)
            
            # Append data to the right list
            features[cfg['mapping'][trigger[-1]]].append(psd)
        
   
            
           
 
for feature in features:
    features[feature] = np.array(features[feature], dtype=np.float32)
    print(feature)
    print(features[feature].shape)

Freq: 100.0
Processing run 1 of 10
Processing run 2 of 10
Processing run 3 of 10
Processing run 4 of 10
Processing run 5 of 10
Processing run 6 of 10
Processing run 7 of 10
Processing run 8 of 10
Processing run 9 of 10
Processing run 10 of 10
thumb
(50, 153, 15, 12)
index
(50, 153, 15, 12)
middle
(50, 153, 15, 12)
ring
(50, 153, 15, 12)
little
(50, 153, 15, 12)


The `psd_features` variable is a 3-dimensional NumPy array that contains the extracted features from the EEG data. The dimensions of the array are `(num_channels, num_windows, num_features)`, where:

1. `num_channels`: The number of EEG channels in your input data.
2. `num_windows`: The number of sliding windows generated from your input data. This number depends on the `window_size` and `step_size` parameters you set, as well as the total number of samples in the input data.
3. `num_features`: The number of features extracted per channel per window. In this example, we are extracting four features: the mean and standard deviation of the PSD for both the Mu (8-12 Hz) and Beta (13-30 Hz) bands.

The content of `psd_features` can be understood as follows:

- `psd_features[ch, win, 0]`: The mean PSD for the Mu band (8-12 Hz) for channel `ch` in window `win`.
- `psd_features[ch, win, 1]`: The standard deviation of the PSD for the Mu band (8-12 Hz) for channel `ch` in window `win`.
- `psd_features[ch, win, 2]`: The mean PSD for the Beta band (13-30 Hz) for channel `ch` in window `win`.
- `psd_features[ch, win, 3]`: The standard deviation of the PSD for the Beta band (13-30 Hz) for channel `ch` in window `win`.

By analyzing the distribution of power in both the Mu and Beta frequency bands separately, you can obtain more detailed insights about the neural activity related to finger movements.


In [80]:
target_dir = 'features'
tag = 'gpt4'
# Save the dictionary to an HDF5 file
file_path = os.path.join(target_dir, tag + '_' +cfg['subjects'][subject_id] + '.h5')
with h5py.File(file_path, 'w') as h5file:
    for key, value in features.items():
        h5file.create_dataset(key, data=value)
print(f'Features saved to {file_path}.')

Features saved to features\gpt4freq_all_S1.h5.
